# Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества F1 не меньше 0.75.

In [6]:
!pip install -q spacy catboost

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from tqdm.notebook import tqdm
tqdm.pandas()
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'nltk'

In [8]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

display(data.head())
data.info()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [9]:
#удалим неинформативный столбец со старыми индексами
data.drop(data.columns[0], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [10]:
"""def lemmatize(text):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    return text"""

def clear_text(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip().lower()
    text = " ".join(text.split())
    return text

data['text'] = data['text'].apply(lambda x: clear_text(x))


lemm_texts = []
total = data.shape[0]
nlp_pipe = nlp.pipe(data['text'].values, disable = ['ner', 'parser'])

for doc in tqdm(nlp_pipe, total=total):
    lemm_text = " ".join([token.lemma_ for token in doc])    
    lemm_texts.append(lemm_text) 

  0%|          | 0/159292 [00:00<?, ?it/s]

In [11]:
stopwords = list(set(nltk_stopwords.words('english')))

## Разделим на выборки

In [22]:
target=data['toxic'].values
features = data['text']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size = .1, 
                                                                            random_state = 42)

features_train.shape[0], features_test.shape[0]

(143362, 15930)

In [23]:
data['text'].head()

0    explanation why the edits made under my userna...
1    daww he matches this background colour im seem...
2    hey man im really not trying to edit war its j...
3    more i cant make any real suggestions on impro...
4    you sir are my hero any chance you remember wh...
Name: text, dtype: object

> Проведена предоработка данных.
* Строки приведены в нижний регистр, перед векторизацией из текста были удалены лишние пробелы, символы конца строки, ненужные символы, а также удалены стоп-слова с помощью библиотеки nltk_stopwords.
* Данные разделены на тренировочную и тестовую выборки
* Проведена векторизация текстов используя TF-IDF

# Обучим модели

## CatBoostClassifier

In [ ]:
%%time
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)), 
    ('model', CatBoostClassifier(verbose=0, iterations=10))  
])

parameters = {
    'model__depth': [2, 4, 8], 
    'model__learning_rate': [0.1, 0.4, 0.7, 0.9] 
}

grid_search = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, verbose=True, n_jobs=-1)
grid_search.fit(features_train, target_train)
print('Лучшие гиперпараметры:', grid_search.best_params_)
print('Лучший F1 CatBoostClassifier:', grid_search.best_score_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


## LogisticRegression

In [ ]:
%%time
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),  # TF-IDF векторизация
    ('model', LogisticRegression(random_state=42, solver='liblinear', class_weight="balanced")) 
])

parameters = {
            'model__penalty': ['l1', 'l2'],
            'model__C': list(range(1,15,3))
        }

grid_search = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, verbose=True, n_jobs=-1)
grid_search.fit(features_train, target_train)

print('Лучшие гиперпараметры:', grid_search.best_params_)
print('Лучший F1 LogisticRegression:', grid_search.best_score_)

## RandomForest

In [ ]:
%%time

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),  # TF-IDF векторизация
    ('model', RandomForestClassifier(random_state=42)) 
])

parameters = {
    'model__n_estimators': list(range(50,300,50)),
    'model__max_depth':[5,15],
    'model__max_features' : list(range(1,20, 2))
        }

grid_search = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, verbose=True, n_jobs=-1)
grid_search.fit(features_train, target_train)

print('Лучшие гиперпараметры:', grid_search.best_params_)
print('Лучший F1 RandomForestClassifier:', grid_search.best_score_)

## Проверка на тестовой выборке

In [ ]:
# Проведем Векторизацию даннных тестовой выборки используя TF-IDF
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
features_train = count_tf_idf.fit_transform(features_train)
features_test = count_tf_idf.transform(features_test)

print("Размер матрицы test:", features_test.shape)

In [ ]:
LR = LogisticRegression(random_state=42, C=7, penalty='l2', solver='liblinear', class_weight="balanced")
LR.fit(features_train, target_train)
predictions = LR.predict(features_test)
print('Лучший результат:', f1_score(target_test, predictions))


> Обучены 3 модели:
* CatboostClassifier
* LogisticRegression
* RandomForestClassifier

Лучше всего с поставленной задачей справилась модель LogisticRegression, ее значение F1 на тестовой выборке: 0.77, что выполняет поставленный критерий.

# Общий вывод

В ходе работы над проектом была проведена предоработка данных. Обучена модель классификации комментариев на позитивные и негативные.  Лучше всего с поставленной задачей справилась модель LogisticRegression, ее значение F1 на тестовой выборке: 0.77, что выполняет поставленный критерий.